# Bidirectional LSTM network for emotion detection in twits

- based on: https://aclanthology.org/S19-2034/
- code taken from here: https://github.com/sismetanin/emosense-semeval2019-task3-emocontext

## 1. Loading Data

In [5]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import numpy as np
import time

import io

start = time.time()

label2emotion = {0: "others", 1: "happy", 2: "sad", 3: "angry"}
emotion2label = {"others": 0, "happy": 1, "sad": 2, "angry": 3}

emoticons_additional = {
    '(^・^)': '<happy>', ':‑c': '<sad>', '=‑d': '<happy>', ":'‑)": '<happy>', ':‑d': '<laugh>',
    ':‑(': '<sad>', ';‑)': '<happy>', ':‑)': '<happy>', ':\\/': '<sad>', 'd=<': '<annoyed>',
    ':‑/': '<annoyed>', ';‑]': '<happy>', '(^�^)': '<happy>', 'angru': 'angry', "d‑':":
        '<annoyed>', ":'‑(": '<sad>', ":‑[": '<annoyed>', '(�?�)': '<happy>', 'x‑d': '<laugh>',
}

text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
               'time', 'url', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
              'emphasis', 'censored'},
    fix_html=True,
    segmenter="twitter",
    corrector="twitter",
    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=True,
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons, emoticons_additional]
)


def tokenize(text):
    text = " ".join(text_processor.pre_process_doc(text))
    return text


def preprocessData(dataFilePath, mode):
    conversations = []
    labels = []
    with io.open(dataFilePath, encoding="utf8") as finput:
        finput.readline()
        for line in finput:
            line = line.strip().split('\t')
            for i in range(1, 4):
                line[i] = tokenize(line[i])
            if mode == "train":
                labels.append(emotion2label[line[4]])
            conv = line[1:4]
            conversations.append(conv)
    if mode == "train":
        return np.array(conversations), np.array(labels)
    else:
        return np.array(conversations)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [6]:
texts_train, labels_train = preprocessData('data\\train.txt', mode="train")
texts_dev, labels_dev = preprocessData('data\\dev.txt', mode="train")
texts_test, labels_test = preprocessData('data\\test.txt', mode="train")

## 2. Loading Word Embeddings

In [7]:
def getEmbeddings(file):
    embeddingsIndex = {}
    dim = 0
    with io.open(file, encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            embeddingVector = np.asarray(values[1:], dtype='float32')
            embeddingsIndex[word] = embeddingVector 
            dim = len(embeddingVector)
    return embeddingsIndex, dim


def getEmbeddingMatrix(wordIndex, embeddings, dim):
    embeddingMatrix = np.zeros((len(wordIndex) + 1, dim))
    for word, i in wordIndex.items():
        embeddingMatrix[i] = embeddings.get(word)
    return embeddingMatrix

In [8]:
from keras.preprocessing.text import Tokenizer

embeddings, dim = getEmbeddings('emosense.300d.txt')
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts([' '.join(list(embeddings.keys()))])

wordIndex = tokenizer.word_index
print("Found %s unique tokens." % len(wordIndex))

embeddings_matrix = getEmbeddingMatrix(wordIndex, embeddings, dim) 

Found 658129 unique tokens.


## 3. Texts Tokenization

In [9]:
from keras.utils import to_categorical
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

MAX_SEQUENCE_LENGTH = 24

X_train, X_val, y_train, y_val = train_test_split(texts_train, labels_train, test_size=0.2, random_state=42)

labels_categorical_train = to_categorical(np.asarray(y_train))
labels_categorical_val = to_categorical(np.asarray(y_val))
labels_categorical_dev = to_categorical(np.asarray(labels_dev))
labels_categorical_test = to_categorical(np.asarray(labels_test))


def get_sequences(texts, sequence_length):
    message_first = pad_sequences(tokenizer.texts_to_sequences(texts[:, 0]), sequence_length)
    message_second = pad_sequences(tokenizer.texts_to_sequences(texts[:, 1]), sequence_length)
    message_third = pad_sequences(tokenizer.texts_to_sequences(texts[:, 2]), sequence_length)
    return message_first, message_second, message_third


message_first_message_train, message_second_message_train, message_third_message_train = get_sequences(X_train, MAX_SEQUENCE_LENGTH)
message_first_message_val, message_second_message_val, message_third_message_val = get_sequences(X_val, MAX_SEQUENCE_LENGTH)
message_first_message_dev, message_second_message_dev, message_third_message_dev = get_sequences(texts_dev, MAX_SEQUENCE_LENGTH)
message_first_message_test, message_second_message_test, message_third_message_test = get_sequences(texts_test, MAX_SEQUENCE_LENGTH)

## 3. Bidirectional LSTM 

In [10]:
from keras.layers import Input, Dense, Embedding, Concatenate, \
    Dropout, LSTM, Bidirectional, GaussianNoise
from keras.models import Model


def buildModel(embeddings_matrix, sequence_length, lstm_dim, hidden_layer_dim, num_classes, 
               noise=0.1, dropout_lstm=0.2, dropout=0.2):
    turn1_input = Input(shape=(sequence_length,), dtype='int32')
    turn2_input = Input(shape=(sequence_length,), dtype='int32')
    turn3_input = Input(shape=(sequence_length,), dtype='int32')
    embedding_dim = embeddings_matrix.shape[1]
    embeddingLayer = Embedding(embeddings_matrix.shape[0],
                                embedding_dim,
                                weights=[embeddings_matrix],
                                input_length=sequence_length,
                                trainable=False)
    
    turn1_branch = embeddingLayer(turn1_input)
    turn2_branch = embeddingLayer(turn2_input) 
    turn3_branch = embeddingLayer(turn3_input) 
    
    turn1_branch = GaussianNoise(noise, input_shape=(None, sequence_length, embedding_dim))(turn1_branch)
    turn2_branch = GaussianNoise(noise, input_shape=(None, sequence_length, embedding_dim))(turn2_branch)
    turn3_branch = GaussianNoise(noise, input_shape=(None, sequence_length, embedding_dim))(turn3_branch)

    lstm1 = Bidirectional(LSTM(lstm_dim, dropout=dropout_lstm))
    lstm2 = Bidirectional(LSTM(lstm_dim, dropout=dropout_lstm))
    
    turn1_branch = lstm1(turn1_branch)
    turn2_branch = lstm2(turn2_branch)
    turn3_branch = lstm1(turn3_branch)
    
    x = Concatenate(axis=-1)([turn1_branch, turn2_branch, turn3_branch])
    
    x = Dropout(dropout)(x)
    
    x = Dense(hidden_layer_dim, activation='relu')(x)
    
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=[turn1_input, turn2_input, turn3_input], outputs=output)
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

model = buildModel(embeddings_matrix, MAX_SEQUENCE_LENGTH, lstm_dim=64, hidden_layer_dim=30, num_classes=4) 

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 24, 300)      197439000   ['input_1[0][0]',                
                                                                  'input_2[0][0]',            

In [12]:
from kutilities.callbacks import MetricsCallback
from sklearn.metrics import f1_score, precision_score, recall_score
from keras.callbacks import ModelCheckpoint, TensorBoard

metrics = {
    "f1_e": (lambda y_test, y_pred:
             f1_score(y_test, y_pred, average='micro',
                      labels=[emotion2label['happy'],
                              emotion2label['sad'],
                              emotion2label['angry']
                              ])),
    "precision_e": (lambda y_test, y_pred:
                    precision_score(y_test, y_pred, average='micro',
                                    labels=[emotion2label['happy'],
                                            emotion2label['sad'],
                                            emotion2label['angry']
                                            ])),
    "recall_e": (lambda y_test, y_pred:
                 recall_score(y_test, y_pred, average='micro',
                              labels=[emotion2label['happy'],
                                      emotion2label['sad'],
                                      emotion2label['angry']
                                      ]))
}

filepath = "models\\bidirectional_LSTM_best_weights_{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', save_best_only=True, save_weights_only=False, mode='auto')
tensorboardCallback = TensorBoard(log_dir='Graph', histogram_freq=0, write_graph=True, write_images=True)

In [13]:
history = model.fit([message_first_message_train, message_second_message_train, message_third_message_train],
                    np.array(labels_categorical_train),
                    callbacks=[checkpoint, tensorboardCallback],
                    validation_data=(
                        [message_first_message_val, message_second_message_val, message_third_message_val],
                        np.array(labels_categorical_val)
                    ),
                    epochs=20,
                    batch_size=200)

Epoch 1/20
121/121 [==============================] - 43s 276ms/step - loss: 0.8040 - acc: 0.6696 - val_loss: 0.4088 - val_acc: 0.8442
Epoch 2/20
121/121 [==============================] - 32s 265ms/step - loss: 0.4296 - acc: 0.8396 - val_loss: 0.3245 - val_acc: 0.8876
Epoch 3/20
121/121 [==============================] - 32s 266ms/step - loss: 0.3649 - acc: 0.8658 - val_loss: 0.3008 - val_acc: 0.8922
Epoch 4/20
121/121 [==============================] - 32s 266ms/step - loss: 0.3249 - acc: 0.8818 - val_loss: 0.2823 - val_acc: 0.9020
Epoch 5/20
121/121 [==============================] - 31s 255ms/step - loss: 0.3014 - acc: 0.8907 - val_loss: 0.2649 - val_acc: 0.9096
Epoch 6/20
121/121 [==============================] - 29s 243ms/step - loss: 0.2767 - acc: 0.9006 - val_loss: 0.2599 - val_acc: 0.9072
Epoch 7/20
121/121 [==============================] - 29s 240ms/step - loss: 0.2613 - acc: 0.9059 - val_loss: 0.2491 - val_acc: 0.9090
Epoch 8/20
121/121 [==============================] - 3

In [14]:
print('training time: {:.2f} seconds'.format(time.time() - start))

training time: 725.17 seconds


In [19]:
model.load_weights("models\\bidirectional_LSTM_best_weights_16-0.9199.hdf5")

In [20]:
y_pred = model.predict([message_first_message_dev, message_second_message_dev, message_third_message_dev])

87/87 [==============================] - 3s 32ms/step


In [21]:
from sklearn.metrics import classification_report

for title, metric in metrics.items():
    print(title, metric(labels_categorical_dev.argmax(axis=1), y_pred.argmax(axis=1)))
print(classification_report(labels_categorical_dev.argmax(axis=1), y_pred.argmax(axis=1)))

f1_e 0.7052845528455286
precision_e 0.6119929453262787
recall_e 0.8321342925659473
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      2338
           1       0.55      0.79      0.65       142
           2       0.72      0.82      0.77       125
           3       0.61      0.88      0.72       150

    accuracy                           0.90      2755
   macro avg       0.71      0.85      0.77      2755
weighted avg       0.92      0.90      0.91      2755



## 4. Performance Evaluation

In [22]:
start = time.time()

y_pred = model.predict([message_first_message_test, message_second_message_test, message_third_message_test])

print('prediction time: {:.2f} seconds'.format(time.time() - start))

for title, metric in metrics.items():
    print(title, metric(labels_categorical_test.argmax(axis=1), y_pred.argmax(axis=1)))
print(classification_report(labels_categorical_test.argmax(axis=1), y_pred.argmax(axis=1)))

173/173 [==============================] - 5s 31ms/step
prediction time: 5.52 seconds
f1_e 0.7099433281813499
precision_e 0.6212804328223624
recall_e 0.828125
              precision    recall  f1-score   support

           0       0.97      0.92      0.94      4677
           1       0.59      0.80      0.68       284
           2       0.68      0.84      0.75       250
           3       0.61      0.85      0.71       298

    accuracy                           0.90      5509
   macro avg       0.71      0.85      0.77      5509
weighted avg       0.92      0.90      0.91      5509

